![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Recorders

## What is a `Recorder`?!

A `Recorder` saves results of a [Driver](03-Drivers.ipynb) for data sharing and/or analysis.

## Introduction

### Add a `Recorder`

Simply use the `add_recorder` of a `Driver` method passing a recorder object you want to use 
(see [Available recorders](#Available-recorders) section of this tutorial).

In [ ]:
from cosapp.tests.library.systems import Multiply1
from cosapp.systems import System
from cosapp.drivers import RunOnce
from cosapp.recorders import DataFrameRecorder

m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))
run.add_recorder(DataFrameRecorder());

### Default behaviour

By default, units are added in the headers with the variable name. And some special columns are presented:
- `Section`: User name reference
- `Reference`: Internal reference for the record
- `Status`: Status of the execution for the record
- `Error code`: Error code for the record

In [ ]:
m.run_drivers()
run.recorder.data

### Common options

All recorders share the following options at creation:
- `includes`: Variable name or list of variable names to be recorded. It supports *\** and *?* special characters - the former will be replaced by 0 to infinity of any character and the latter will be replace by *one* character.
- `excludes`: Variable name or list of variable names not to be recorded. It supports *\** and *?* too. `excludes` shadow `includes`.
- `section`: String to named a section of the recording
- `hold`: Boolean to hold previous results or not in the recorder.
- `precision`: Integer specifying the number of digits used when writing floating point number.
- `raw_output`: Don't add units in variable header neither add the special columns

`section`, `hold` and `precision` can be modified after the recorder creation.

In [ ]:
class Example(System):
    
    def setup(self):
        self.add_inward('one', 0.)
        self.add_inward('one_two', [0., 1.])
        self.add_inward('two_1', 'test')
        self.add_inward('two_2', 42.)
        
e = Example('test')
run = e.add_driver(RunOnce('run'))
run.add_recorder(DataFrameRecorder(includes=['one*', 'two_?'], excludes=['one_*', ]))
e.run_drivers()
run.recorder.data

In [ ]:
run.recorder.hold = True
e.one = 22.
e.run_drivers()
run.recorder.data

## Available recorders

### DataFrameRecorder

Record the data in memory to be accessed after the execution through the `inwards` property as a
`pandas.DataFrame`.

### DSVRecorder

Record the data in a Delimited Separated Value formatted file. The constructor requires the 
`filepath`. And it has two specific options:
- `delimiter`: Delimiter to use (possibilities are `,` , `;` or `\t`)
- `buffer`: Boolean specifiying if the data must be written during the execution (`buffer=False`)
or after it (`buffer=True`). The default behaviour is `buffer=False`.

In [ ]:
from cosapp.recorders import DSVRecorder
m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))
run.add_recorder(DSVRecorder('test.csv', delimiter=';', precision=2))
m.run_drivers()

In [ ]:
%less test.csv